In [ ]:
import os
import re
import io
import sys
import glob
import enum
import json
import dask
import requests
import datetime
import numpy as np
import pandas as pd

import dask.diagnostics
import sqlalchemy as db
import sqlalchemy.orm
import sqlalchemy.ext.declarative
from matplotlib import pyplot as plt

sys.path.append('../..')

%load_ext autoreload
%autoreload 1
%aimport opencell.database.uniprot_utils

from opencell.database import models, uniprot_utils
from opencell.database import operations as ops
from opencell.database import utils as db_utils

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-dev.json')
# url = db_utils.url_from_credentials('../../db-credentials-cap.json')
url

In [ ]:
engine = db.create_engine(url)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)

### Download and cache metadata from UniprotKB

In [ ]:
# all target_names
designs = Session.query(models.CrisprDesign).all()
len(designs)

In [ ]:
uniprot_utils.get_uniprot_metadata(gene_name=designs[111].target_name)

In [ ]:
uniprot_utils.query_uniprotkb('CAD', limit=2)

In [ ]:
# retrieve and concatenate metadata for all crispr designs
tasks = [
    dask.delayed(uniprot_utils.query_uniprotkb)(design.transcript_id, limit=1) 
    for design in designs
]

with dask.diagnostics.ProgressBar():
    rows = dask.compute(*tasks)

metadata = pd.concat(tuple(rows), axis=0)

In [ ]:
metadata.to_csv('/Users/keith.cheveralls/Downloads/2020-05-21_top-uniprotKB-hit-for-all-targets.csv')

In [ ]:
# check for rows without a uniprot_id
metadata.loc[metadata.uniprot_id.isna()]

In [ ]:
# retrieve and insert uniprot metadata for all designs
@dask.delayed
def task(Session, design_id):
    ops.insert_uniprot_metadata(Session(), design_id)

designs = Session.query(models.CrisprDesign).all()
tasks = [task(Session, design.id) for design in designs]

with dask.diagnostics.ProgressBar():
    dask.compute(*tasks)